## Context Manager

* https://peps.python.org/pep-0343/
* enter, exit
* resource setup and teardown

In [1]:
with open("ContextManager.ipynb") as f:
    print(len(f.read()))

72


Use cases:

* files
* temporary files
* database connections
* timing
* temporary state changes
* ...

In [3]:
class A:
    def __enter__(self):
        print("enter")
    
    def __exit__(self, exc, exc_type, traceback):
        print("exit")

> **Enter** the runtime context and return either this object or another object related to the runtime context. The value returned by this method is bound to the identifier in the as clause of with statements using this context manager.

> **Exit** the runtime context and return a Boolean flag indicating if any exception that occurred should be suppressed. If an exception occurred while executing the body of the with statement, the arguments contain the exception type, value and traceback information. Otherwise, all three arguments are None.

In [5]:
with A():
    print("c")

enter
c
exit


Other examples:
    
* [sqlite](https://github.com/miku/siskin/blob/469f973addd20f4d40820aaa0f1c4b47659f5900/siskin/database.py#L36-L59)

Returning a value

In [8]:
class A:
    def __enter__(self):
        return "any"
    
    def __exit__(self, exc, exc_type, traceback):
        print("exit")

In [9]:
with A() as v:
    print(v)

any
exit


### Exceptions

Returning a true value from this method will cause the with statement to suppress the exception and continue execution with the statement immediately following the with statement. Otherwise the exception continues propagating after this method has finished executing. Exceptions that occur during execution of this method will replace any exception that occurred in the body of the with statement.

In [10]:
class A:
    def __enter__(self):
        raise RuntimeError("enter")
        return "any"
    
    def __exit__(self, exc, exc_type, traceback):
        print("exit")

In [11]:
with A() as v:
    print(v)

RuntimeError: enter

In [26]:
class A:
    def __enter__(self):
        return "any"
    
    def __exit__(self, exc, exc_type, traceback):
        print("exit", exc, exc_type, traceback)
        return True # True # False

In [27]:
with A() as v:
    raise RuntimeError("with")
    print(v)

exit <class 'RuntimeError'> with <traceback object at 0x7f14920f7a40>


### Using a generator

In [30]:
from contextlib import contextmanager

@contextmanager
def managed_resource(*args, **kwds):
    # Code to acquire resource, e.g.:
    print("setting up")
    try:
        yield 123
    finally:
        # Code to release resource, e.g.:
        print("cleaning up")

In [31]:
with managed_resource() as v:
    print(v)

setting up
123
cleaning up


* misc helpers in context lib

In [34]:
import contextlib, os

In [35]:
with contextlib.suppress(FileNotFoundError):
    os.remove('someotherfile.tmp')

### Dual use context manager and decorator

In [36]:
from contextlib import ContextDecorator

class mycontext(ContextDecorator):
    def __enter__(self):
        print('Starting')
        return self

    def __exit__(self, *exc):
        print('Finishing')
        return False

In [38]:
@mycontext()
def f():
    print('The bit in the middle')

f()

Starting
The bit in the middle
Finishing


In [39]:
with mycontext():
    print('The bit in the middle')


Starting
The bit in the middle
Finishing


**Task**